In [4]:
import os

In [5]:
%pwd

'e:\\Trekathon\\Early-Fish-Disease-Detection\\notebooks'

In [6]:
os.chdir("../")

In [7]:
%pwd

'e:\\Trekathon\\Early-Fish-Disease-Detection'

In [8]:
from dataclasses import dataclass
from pathlib import Path

In [9]:
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [10]:
from early_fish_disease_detection.constants import *
from early_fish_disease_detection.utils.common import read_yaml, create_dir
import tensorflow as tf

In [11]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_dir([self.config.artifacts_root])

    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_dir([Path(model_ckpt_dir), Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Freshwater Fish Disease Aquaculture in south asia")
        create_dir([Path(training.root_dir)])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config


In [12]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [13]:
class PrepareCallbacks:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [14]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )


    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
        )

        # The size of images and batch size
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        # Create the ImageDataGenerator for validation and training data
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        # Load the validation data from the 'test' directory
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data / 'test',  # Path to your 'test' folder
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            # Apply augmentation to training data
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )
        else:
            # If no augmentation, use same as validation
            train_datagenerator = valid_datagenerator

        # Load the training data from the 'train' directory
        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data / 'train',  # Path to your 'train' folder
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)


    def convert_to_tflite(self):
        converter = tf.lite.TFLiteConverter.from_keras_model(self.model)

        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        
        tflite_model = converter.convert()

        tflite_model_path = self.config.trained_model_path.with_suffix(".tflite")
        with open(tflite_model_path, 'wb') as f:
            f.write(tflite_model)
        print(f"Model successfully converted to TFLite and saved at {tflite_model_path}")


    
    def train(self, callback_list: list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=callback_list
        )

        # Save the trained model
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

        # Convert the model to TFLite after training
        self.convert_to_tflite()

In [15]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)

except Exception as e:
    raise e

2025-01-08 12:09:41,737 - Early-Fish-Disease-Detection-logger - INFO - yaml file config\config.yaml read successfully
2025-01-08 12:09:41,742 - Early-Fish-Disease-Detection-logger - INFO - yaml file params.yaml read successfully
2025-01-08 12:09:41,744 - Early-Fish-Disease-Detection-logger - INFO - Directory artifacts created successfully
2025-01-08 12:09:41,747 - Early-Fish-Disease-Detection-logger - INFO - Directory artifacts\prepare_callbacks\checkpoint_dir created successfully
2025-01-08 12:09:41,749 - Early-Fish-Disease-Detection-logger - INFO - Directory artifacts\prepare_callbacks\tensorboard_log_dir created successfully
2025-01-08 12:09:41,752 - Early-Fish-Disease-Detection-logger - INFO - Directory artifacts\training created successfully
Found 697 images belonging to 7 classes.


c:\Users\ronit\anaconda3\envs\venv\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Found 1747 images belonging to 7 classes.


c:\Users\ronit\anaconda3\envs\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


109/109 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3654 - loss: 10.0536

c:\Users\ronit\anaconda3\envs\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


109/109 ━━━━━━━━━━━━━━━━━━━━ 229s 2s/step - accuracy: 0.3663 - loss: 10.0365 - val_accuracy: 0.7078 - val_loss: 2.7966
2025-01-08 12:13:48,815 - absl - INFO - Function `function` contains input name(s) resource with unsupported characters which will be renamed to functional_1_dense_1_biasadd_readvariableop_resource in the SavedModel.
2025-01-08 12:13:49,291 - absl - INFO - Function `function` contains input name(s) resource with unsupported characters which will be renamed to functional_1_dense_1_biasadd_readvariableop_resource in the SavedModel.
2025-01-08 12:14:03,095 - tensorflow - INFO - Assets written to: C:\Users\ronit\AppData\Local\Temp\tmpvz7xir7r\assets
Saved artifact at 'C:\Users\ronit\AppData\Local\Temp\tmpvz7xir7r'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 7), dtype=tf.float32, name=None)
Captures:
  2454865439120: 